In [1]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-MiniLM-L6-v2")
print("Loaded model:", model)


C:\Users\HP\Desktop\llm-roadmap\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\HP\Desktop\llm-roadmap\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In orde

Loaded model: SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


In [2]:
sentences = [
    "I am learning LLM engineering.",
    "Next.js and Python make a powerful stack.",
    "Vector search finds similar meanings, not exact words.",
    "I want a remote job building AI SaaS products.",
    "Football is my favorite weekend sport."
]

# Compute embeddings for all sentences
embeddings = model.encode(sentences, convert_to_tensor=True)

print("Number of sentences:", len(sentences))
print("Embedding shape:", embeddings.shape)  # [num_sentences, embedding_dim]
print("First 5 numbers of first embedding:", embeddings[0][:5])


Number of sentences: 5
Embedding shape: torch.Size([5, 384])
First 5 numbers of first embedding: tensor([ 0.0154, -0.0332,  0.0306, -0.0311,  0.0312])


In [3]:
from sentence_transformers import util

query = "I want an AI job working on SaaS."
query_emb = model.encode(query, convert_to_tensor=True)

# Cosine similarity between query and all sentence embeddings
scores = util.cos_sim(query_emb, embeddings)[0]  # shape: [5]

# Sort sentences by similarity score (highest first)
results = sorted(
    enumerate(scores),
    key=lambda x: x[1],
    reverse=True
)

print("Query:", query)
print("\nMost similar sentences:\n")
for idx, score in results:
    print(f"Score: {score:.4f} | Sentence: {sentences[idx]}")


Query: I want an AI job working on SaaS.

Most similar sentences:

Score: 0.8170 | Sentence: I want a remote job building AI SaaS products.
Score: 0.2888 | Sentence: I am learning LLM engineering.
Score: 0.1321 | Sentence: Next.js and Python make a powerful stack.
Score: 0.1088 | Sentence: Vector search finds similar meanings, not exact words.
Score: 0.0070 | Sentence: Football is my favorite weekend sport.


In [4]:
import chromadb
from chromadb.config import Settings

# Create a local Chroma client (stores files in ./chroma_db folder)
client = chromadb.Client(
    Settings(
        anonymized_telemetry=False,
        persist_directory="chroma_db"
    )
)

collection = client.get_or_create_collection("study_sentences")

# Store our 5 sentences + their embeddings
collection.add(
    ids=[f"sent-{i}" for i in range(len(sentences))],
    documents=sentences,
    embeddings=embeddings.tolist()  # convert tensor -> Python list
)

print("Docs in collection:", collection.count())


Docs in collection: 5


In [5]:
query = "How do I get hired to build AI SaaS platforms?"
query_emb = model.encode(query).tolist()

results = collection.query(
    query_embeddings=[query_emb],
    n_results=3
)

print("Query:", query)
print()

for doc, dist in zip(results["documents"][0], results["distances"][0]):
    print(f"Distance: {dist:.4f} | Doc: {doc}")


Query: How do I get hired to build AI SaaS platforms?

Distance: 0.4972 | Doc: I want a remote job building AI SaaS products.
Distance: 1.4945 | Doc: I am learning LLM engineering.
Distance: 1.7103 | Doc: Next.js and Python make a powerful stack.
